In [1]:
%cd ~/Research/baldNER
%load_ext autoreload
%autoreload 2

/home/minh/Research/baldNER


In [2]:
import os
from src.data import conll2003_utils as data_utils

In [3]:
DATA_TRAIN_PATH = "data/raw/CoNLL2003/eng.train"

In [4]:
DATA_PROCESSED_DIR = "data/processed/CoNLL2003/"

In [5]:
sentences = data_utils.load_raw_dataset(DATA_TRAIN_PATH)

In [6]:
# generate vocab set
vocab = data_utils.generate_vocab_set(sentences)

In [7]:
vocab.save(os.path.join(DATA_PROCESSED_DIR, "vocab_idx.txt"))

In [18]:
# generate tags
tags = data_utils.generate_NER_tag_set(sentences)

In [19]:
tags.save(os.path.join(DATA_PROCESSED_DIR, "tags_idx.txt"))

# generating word2vec on data

In [10]:
from src.parameters import WORD_EMBEDDING_SIZE

In [11]:
# processing coNLL data to list of raw sentences, 1 per line
# TODO do we need to lowercase the words and strip symbols
RAW_SENTENCE_FNAME =  os.path.join(DATA_PROCESSED_DIR, "eng.train.raw_sentences.txt")
raw_sentences = []
with open(RAW_SENTENCE_FNAME, 'w') as f:
    for sentence in sentences:
        raw_sentence = " ".join([data['word'] for data in sentence])
        f.write(raw_sentence + "\n")

In [12]:
from gensim.models.word2vec import Word2Vec, LineSentence

In [13]:
gensim_sentences = LineSentence(RAW_SENTENCE_FNAME)

In [14]:
# using CBOW
# negative is negative sampling (https://towardsdatascience.com/nlp-101-negative-sampling-and-glove-936c88f3bc68)
model = Word2Vec(gensim_sentences, sg=1, size=WORD_EMBEDDING_SIZE, workers=4, iter=8, negative=8)

In [15]:
word_vectors = model.wv

In [17]:
word_vectors.save_word2vec_format(
    os.path.join(DATA_PROCESSED_DIR, "word2vec.txt"),
    fvocab=os.path.join(DATA_PROCESSED_DIR, "word2vec_vocab.txt"))

In [33]:
import numpy as np
np.save(os.path.join(DATA_PROCESSED_DIR, "word2vec.vector.npy"), word_vectors.vectors)

In [31]:
from gensim.models.keyedvectors import KeyedVectors

In [32]:
wv = KeyedVectors.load_word2vec_format(os.path.join(DATA_PROCESSED_DIR, "word2vec.txt"))

UnpicklingError: invalid load key, '4'.